In [ ]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

import sys
import traceback

from bs4 import BeautifulSoup
from random import choice
import xlsxwriter

# Libraries required to limit the time taken by a request
import signal
from contextlib import contextmanager

class TimeoutException(Exception): pass

In [ ]:
def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }
    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    df["Date"] = df["Date"].astype(str)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    #table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    table_rows = tree.xpath("//div[contains(@class, 'fin-row')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.

    try:
      assert len(table_rows) > 0

      df = parse_rows(table_rows)
      df = clean_data(df)
    except AttributeError:
      print("Code not found : "+code)
      return
    except IndexError:
      print("Financials not found : "+code)
      return
    except AssertionError: 
      # https://stackoverflow.com/questions/11587223/how-to-handle-assertionerror-in-python-and-find-out-which-line-or-statement-it-o
      _, _, tb = sys.exc_info()
      traceback.print_tb(tb) # Fixed format
      tb_info = traceback.extract_tb(tb)
      filename, line, func, text = tb_info[-1]

      print('An error occurred on line {} in statement {}'.format(line, text))
      return

    return df

def scrape(symbol):
    print('Attempting to scrape data for ' + symbol)

    df_balance_sheet = scrape_table(
        'https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol
        )
    df_income_statement = scrape_table(
        'https://finance.yahoo.com/quote/' + symbol + '/financials?p=' + symbol
        )
    df_cash_flow = scrape_table(
        'https://finance.yahoo.com/quote/' + symbol + '/cash-flow?p=' + symbol
        )

    try: 
      df_balance_sheet = df_balance_sheet.set_index('Date')    
      df_income_statement = df_income_statement.set_index('Date')    
      df_cash_flow = df_cash_flow.set_index('Date')
    except: 
      return None
    
    df_joined = df_balance_sheet \
        .join(df_income_statement, on='Date', how='outer', rsuffix=' - Income Statement') \
        .join(df_cash_flow, on='Date', how='outer', rsuffix=' - Cash Flow') \
        .dropna(axis=1, how='all') \
        .reset_index()
            
    df_joined.insert(1, 'Symbol', symbol)
    
    return df_joined

def scrape_multi(symbols):
    df_combined = pd.concat([scrape(symbol) for symbol in symbols], sort=False)
    df_combined.dropna(subset = ['index'])
    df_combined = df_combined.loc[df_combined["Date"] != "ttm"]
    df_combined.drop(columns=['index'])
    df_combined['Date'] = df_combined['Date'].dt.date
    # df_combined["Date"] = pd.to_datetime(df_combined["Date"], dayfirst = True)
    df_combined.reset_index()

    return df_combined


In [ ]:
df = pd.read_csv("SESprice.dat", 
                 sep=";",  
                 usecols=[2,11,14], 
                 names=['Status','Board','Symbols'])
#trim the leading and trailing spaces of the columns
data_frame_trimmed = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
#filter only the mainboard listed companies & not suspended & not delisted
data_frame_trimmed = data_frame_trimmed.loc[(data_frame_trimmed["Board"] == "MAINBOARD") & (data_frame_trimmed["Status"] != "SUSP") & (data_frame_trimmed["Status"] != "DL") ]
data_frame_trimmed['Symbols'] = data_frame_trimmed['Symbols'].astype(str) + '.SI'
symbols = data_frame_trimmed['Symbols'].tolist()
df_combined = scrape_multi(symbols)
df_combined.info()

In [ ]:
date = datetime.today().strftime('%Y-%m-%d')
writer = pd.ExcelWriter('Yahoo-Finance-Scrape-' + date + '.xlsx')
df_combined.to_excel(writer, index = False)
writer.save()